In [ ]:
# from __future__ import division

In [53]:
import psycopg2
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
import re
# from __future__ import division
from nltk.tag import StanfordNERTagger

In [54]:
conn = psycopg2.connect("dbname='cap' user='postgres' host='ec2-35-163-99-253.us-west-2.compute.amazonaws.com' port=9000 password ='secret'")
cursor = conn.cursor()
cursor.execute("SELECT * FROM articles limit 1000")
data = cursor.fetchall()
df = pd.DataFrame(data)
df.columns = ['site', 'title','author','secondary_authors','published_on','accessed_on','url','body' ,'html','newspaper_keywords','newspaper_summary','id']                 
#print(records)

In [55]:
# conn = psycopg2.connect("dbname='cap' user='postgres' host='ec2-35-163-99-253.us-west-2.compute.amazonaws.com' port=9000 password ='secret'")
# df = pd.read_sql_query("SELECT * FROM article_dim limit 5", conn)

In [56]:
df

,site,title,author,secondary_authors,published_on,accessed_on,url,body,html,newspaper_keywords,newspaper_summary,id
0,Huffington Post,Attentat de Manchester: continuons à assumer n...,None,[],2017-05-23,2017-05-23 11:55:45.507713,http://quebec.huffingtonpost.ca/2017/05/23/att...,ADVERTISEMENT Il faut continuer à vivre et à ...,"<div><p class=""sharebox-label"">ADVERTISEMENT</...","{sur,une,la,et,mode,assumer,vie,du,à,dit,couil...",ADVERTISEMENTIl faut continuer à vivre et à as...,76696
1,Huffington Post,Teenager Georgina Callander Named As First Vic...,Luke Cooper,"['Associate Editor', 'Huffpost Australia']",2017-05-23,2017-05-23 11:56:01.384819,http://www.huffingtonpost.com.au/2017/05/23/fi...,Georgina Callander had tweeted her excitement ...,<div><p>Georgina Callander had tweeted her exc...,"{incident,teenager,confirmed,attack,named,call...",Georgina Callander had tweeted her excitement ...,76697
2,Huffington Post,Donald Trump Says Israel And Neighbours Can Un...,None,[],2017-05-22,2017-05-23 11:56:06.908213,http://www.huffingtonpost.ca/2017/05/22/donald...,ADVERTISEMENT JERUSALEM — President Donald Tr...,"<div><p class=""sharebox-label"">ADVERTISEMENT</...","{israel,president,israeli,trumps,cause,unite,i...",ADVERTISEMENTJERUSALEM — President Donald Trum...,76698
3,Huffington Post,23. Mai - der vergessene Verfassungstag,None,[],2017-05-22,2017-05-23 11:56:10.943383,http://www.huffingtonpost.de/christian-wolff/d...,"Wenn es nach Johannes Rau, damals Ministerpräs...","<div><p class=""ad_share_box""> ...","{das,ist,nicht,und,als,grundgesetz,die,mai,ver...",Mai 1949 das Grundgesetz der Bundesrepublik De...,76699
4,Huffington Post,Cet artiste pose des condoms sur les graffitis...,None,[],2017-05-22,2017-05-23 11:56:14.158863,http://quebec.huffingtonpost.ca/2017/05/22/art...,ADVERTISEMENT Les villes pullulent de graffit...,"<div><p class=""sharebox-label"">ADVERTISEMENT</...","{sur,que,sensibiliser,condoms,il,la,des,pose,i...",On ne devrait pas voir ça à tous les coins de ...,76700
5,Huffington Post,Pactes électoraux: le PQ doit assumer ses resp...,None,[],2017-05-22,2017-05-23 11:56:31.031724,http://quebec.huffingtonpost.ca/2017/05/22/pac...,ADVERTISEMENT Le Parti québécois (PQ) doit re...,"<div><p class=""sharebox-label"">ADVERTISEMENT</...","{que,électoraux,québec,pq,la,et,des,vidéo,les,...",ADVERTISEMENTLe Parti québécois (PQ) doit reco...,76701
6,Huffington Post,여성들이 취중에 술집 화장실에서 겪은 최고의 경험담들을 공유했다,None,[],2017-05-22,2017-05-23 11:57:09.267127,http://www.huffingtonpost.kr/2017/05/22/story_...,Portrait of a three young woman partying in a ...,<div><p>Portrait of a three young woman partyi...,"{했다,여성들이,최고의,한,술집,있었다,나는,경험담들을,겪은,화장실에서,취한,화장실...",엄청나게 취한 영국 여성이 내게 와서 내 얼굴을 잡고 “너는 가장 밝게 빛나는 보석...,76702
7,Huffington Post,Healthy Pets Sometimes Euthanized For Owner's ...,None,[],2017-05-22,2017-05-23 11:57:21.779893,http://www.huffingtonpost.ca/2017/05/22/health...,ADVERTISEMENT HALIFAX — Animal advocates are ...,"<div><p class=""sharebox-label"">ADVERTISEMENT</...","{euthanized,animals,vets,animal,convenience,ve...",ADVERTISEMENTHALIFAX — Animal advocates are ca...,76703
8,CNET,"Hipster photographers rejoice, this phone was ...",None,[],None,2017-05-23 12:05:40.744470,https://www.cnet.com/news/kodak-ektra-hipster-...,Patrick Holland/CNET If you want a Kodak Ektr...,"<div><span><span><img src=""https://cnet1.cbsis...","{rejoice,hipster,phone,works,ektra,best,androi...","Patrick Holland/CNETIf you want a Kodak Ektra,...",76704
9,CNET,"Facebook deals with 54,000 cases of porn -- a ...",None,[],None,2017-05-23 12:05:45.022439,https://www.cnet.com/news/facebook-deals-with-...,"Jaap Arriens, NurPhoto via Getty Images Faceb...","<div><span><span><img src=""https://cnet3.cbsis...","{54000,worlds,content,facebook,deals,porn,chal...",The world's largest social network deals with ...,76705


In [3]:
tokenized_body = []
for body in df['body']:
    body = body.decode('utf-8')
    tokens = nltk.word_tokenize(body)
    tokenized_body.append(tokens)

In [4]:
se = pd.Series(tokenized_body)
df['tokenized_body'] = se.values

In [19]:
word_count = []
for body in df['tokenized_body']:
    word_count.append(len(body))

In [20]:
se = pd.Series(word_count)
df['word_count'] = se.values

In [21]:
stop_words = stopwords.words('english')
stop_words = stop_words + [',', '.', '!', '?', '"','\'', '/', '\\', '-', '--', '—', '(', ')', '[', ']', '\'s', '\'t', '\'ve', '\'d', '\'ll', '\'re']
stop_words = set(stop_words) # making this a set increases performance for large documents

In [34]:
df

,site,title,author,secondary_authors,published_on,accessed_on,url,body,html,newspaper_keywords,newspaper_summary,id,tokenized_body,word_count,stopworded_body,lemmatized_body,word_bag
0,NBC,O.J. Simpson to Receive Parole Hearing in July...,Andrew Blankstein,['Daniella Silva'],None,2017-05-23 09:33:29.288780,http://www.nbcnews.com/news/us-news/o-j-simpso...,"After nearly a decade behind bars, O.J. Simpso...","<div><p> After nearly a decade behind bars, O....","{simpson,board,charges,nevada,oj,prison,receiv...",Simpson is set to face a parole hearing this J...,75718,"[After, nearly, a, decade, behind, bars, ,, O....",438,"[after, nearly, decade, behind, bars, o.j, sim...","[after, nearly, decade, behind, bar, o.j, simp...","[(parole, 12), (simpson, 11), (charge, 5), (bo..."
1,NBC,Justice Department Narrows Possible Sanctions ...,Pete Williams,[],None,2017-05-23 09:33:45.267153,http://www.nbcnews.com/news/us-news/justice-de...,Attorney General Jeff Sessions said Monday tha...,<div><p> Attorney General Jeff Sessions said M...,"{order,possible,justice,narrows,sanctions,secu...",Attorney General Jeff Sessions said Monday tha...,75719,"[Attorney, General, Jeff, Sessions, said, Mond...",366,"[attorney, general, jeff, sessions, said, mond...","[attorney, general, jeff, session, say, monday...","[(local, 6), (federal, 6), (city, 6), (session..."
2,NBC,Daughter Pleads for Help in Search for Missing...,Rachael Trost,[],None,2017-05-23 09:33:54.960824,http://www.nbcnews.com/feature/missing-in-amer...,Clarissa is Lisa Harvey's only child. They tw...,<div><p> Clarissa is Lisa Harvey's only child....,"{lake,woman,phone,daughter,told,missing,lisas,...",Lisa Harvey Help Find Lisa Gail Harvey Faceboo...,75720,"[Clarissa, is, Lisa, Harvey, 's, only, child, ...",991,"[clarissa, lisa, harvey, child, they, two, inc...","[clarissa, lisa, harvey, child, they, two, inc...","[(lisa, 26), (clarissa, 18), (say, 14), ('', 1..."
3,NBC,Alabama Death Row Inmate Tommy Arthur Pleads f...,Tracy Connor,[],None,2017-05-23 09:34:07.816292,http://www.nbcnews.com/storyline/lethal-inject...,"Seven times, Tommy Arthur has escaped death. W...","<div><p> Seven times, Tommy Arthur has escaped...","{row,inmate,court,wicker,guilty,testing,arthur...","Seven times, Tommy Arthur has escaped death.\n...",75721,"[Seven, times, ,, Tommy, Arthur, has, escaped,...",1221,"[seven, times, tommy, arthur, escaped, death, ...","[seven, time, tommy, arthur, escape, death, wi...","[(arthur, 27), (``, 17), ('', 16), (say, 15), ..."
4,NBC,Did Ford Nix Its CEO Because He Was Too Much o...,Paul A. Eisenstein,[],None,2017-05-23 09:34:47.090816,http://www.nbcnews.com/business/autos/ford-s-f...,Former Ford CEO Mark Fields fashioned himself ...,<div><p> Former Ford CEO Mark Fields fashioned...,"{visionary,hackett,vision,nix,ceo,chairman,for...",Former Ford CEO Mark Fields fashioned himself ...,75722,"[Former, Ford, CEO, Mark, Fields, fashioned, h...",1247,"[former, ford, ceo, mark, fields, fashioned, a...","[former, ford, ceo, mark, field, fashion, agen...","[(ford, 30), (field, 19), (``, 11), ('', 11), ..."


In [22]:
stopworded_body = []
for body in df['tokenized_body']:
    stopworded_body.append([w.lower() for w in body if w not in stop_words])

In [23]:
se = pd.Series(stopworded_body)
df['stopworded_body'] = se.values

In [24]:
wnl = nltk.WordNetLemmatizer()
lemmatized_words = []
lemmatized_body = []
for body in df['stopworded_body']:
    # We need to tag words with their parts of speech before the WordNet lemmatizer will work properly
    pos_tagged_body = nltk.pos_tag(body)
    lemmatized_words = []
    for word, tag in pos_tagged_body:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = wnl.lemmatize(word, wntag)
        lemmatized_words.append(lemma)
    lemmatized_body.append(lemmatized_words)

In [25]:
se = pd.Series(lemmatized_body)
df['lemmatized_body'] = se.values

In [26]:
word_bag = []
for body in df['lemmatized_body']:
    fdist = FreqDist(body)
    # FreqDist returns a special nltk.probability.FreqDist type
    # This is a list of tuples
    # Here is an example of how to access the elements for future reference
#     print(fdist.most_common())
    # Access an individual tuple
#     print(fdist.most_common()[0])
    # Access the word from the tuple
#     print(fdist.most_common()[0][0])
    # Access the count from the tuple
#     print(fdist.most_common()[0][1])
    # Append to list as ordered frequency distribution
    word_bag.append(fdist.most_common())


In [27]:
se = pd.Series(word_bag)
df['word_bag'] = se.values

In [28]:
st = StanfordNERTagger('/media/justin/Data/Google Drive/Assignments and Projects/Machine Learning/NLP/english.all.3class.distsim.crf.ser.gz',
					   '/media/justin/Data/Google Drive/Assignments and Projects/Machine Learning/NLP/stanford-ner.jar',
					   encoding='utf-8')

In [29]:
classified_texts = []
for body in df['tokenized_body']:
    classified_texts.append(st.tag(body))

# print(classified_text)

In [30]:
from nltk import pos_tag
from nltk.chunk import conlltags2tree
from nltk.tree import Tree

def stanfordNE2BIO(tagged_sent):
    bio_tagged_sent = []
    prev_tag = "O"
    for token, tag in tagged_sent:
        if tag == "O": #O
            bio_tagged_sent.append((token, tag))
            prev_tag = tag
            continue
        if tag != "O" and prev_tag == "O": # Begin NE
            bio_tagged_sent.append((token, "B-"+tag))
            prev_tag = tag
        elif prev_tag != "O" and prev_tag == tag: # Inside NE
            bio_tagged_sent.append((token, "I-"+tag))
            prev_tag = tag
        elif prev_tag != "O" and prev_tag != tag: # Adjacent NE
            bio_tagged_sent.append((token, "B-"+tag))
            prev_tag = tag

    return bio_tagged_sent

In [31]:
def stanfordNE2tree(ne_tagged_sent):
    bio_tagged_sent = stanfordNE2BIO(ne_tagged_sent)
    sent_tokens, sent_ne_tags = zip(*bio_tagged_sent)
    sent_pos_tags = [pos for token, pos in pos_tag(sent_tokens)]

    sent_conlltags = [(token, pos, ne) for token, pos, ne in zip(sent_tokens, sent_pos_tags, sent_ne_tags)]
    ne_tree = conlltags2tree(sent_conlltags)
    return ne_tree


In [40]:
ne_trees = []
for text in classified_texts:
    try:
        ne_trees.append(stanfordNE2tree(text))
    except:
        ne_trees.append(' ')

In [41]:
ne_in_sent = []
ne_in_sents = []
for tree in ne_trees:
    ne_in_sent = []
    for subtree in tree:
        if type(subtree) == Tree: # If subtree is a noun chunk, i.e. NE != "O"
            ne_label = subtree.label()
            ne_string = " ".join([token for token, pos in subtree.leaves()])
            ne_in_sent.append((ne_string, ne_label))
    ne_in_sents.append(ne_in_sent)

In [42]:
se = pd.Series(ne_in_sents)
df['named_entities'] = se.values

In [43]:
def lexical_diversity(text):
    return len(set(text)) / len(text) * 100

In [44]:
lex_div = []
for body in df['stopworded_body']:
    lex_div.append(lexical_diversity(body))

In [45]:
se = pd.Series(lex_div)
df['lexical_diversity'] = se.values

In [48]:
!jupyter nbconvert --to script /home/justin/GitHub/CapstoneI/config_template.ipynb

[NbConvertApp] WARNING | pattern u'/home/justin/GitHub/CapstoneI/config_template.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdout
    Write notebook output to stdout instead of files.
--debug
    set log level to logging.DEBUG (maximize logging output)
--inplace
    Run nbconvert in place, overwriting the existing notebook (only 
    relevant when converting to n